In [298]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,root_mean_squared_error
import plotly.express as px

In [299]:
df = pd.read_csv("Data4.csv")
df = df.drop("Sl.", axis=1)
# df = df.drop(1, axis=0, inplace=True)   ## For Droping a row with index 1.
df.head()

,x1,x2,x3,y
0,0.1,0.311089,0.390541,14.705876
1,0.2,0.639066,1.389918,26.585100
2,0.3,0.840228,1.939903,36.791509
3,0.4,1.299710,2.153009,34.090806
4,0.5,0.941784,0.945136,31.118089


In [300]:
X0 = [float(1) for i in df["x1"]]
X1 = [float(i) for i in df["x1"]]
X2 = [float(i) for i in df["x2"]]
X3 = [float(i) for i in df["x3"]]
Y = [float(i) for i in df["y"]]

X_list = [X0, X1, X2, X3] 
# len(df.columns)
# len(df)

**This is implimentation of simple linear regression using matrix inverse; For multiple input and one output.**

In [301]:
n = len(df) #100
k = len(df.columns) #4
Vector_Y = []

for i in range(k):

    var = []
    for j in range(n):
        p = X_list[i][j] * Y[j]
        var.append(p)

    Vector_Y.append(sum(var))

print(Vector_Y)


[10441.80045016, 66074.554508718, 71379.27277894279, 76343.5729632174]


In [302]:
Xi = [] # for eg. X1
for i in range(1, k):
    Xi.append(sum(X_list[i]))


XiXi = [] # for eg. X1X1
for i in range(1, k):
    var = []
    for j in range(n):
        p = X_list[i][j] * X_list[i][j]
        var.append(p)
    XiXi.append(sum(var))


odds = []
l = [[1,2], [2,3], [3,1]]

for i in range(len(l)):
    f =l[i][0]
    s =l[i][1]

    ll = []
    for j in range(n):
        p = X_list[f][j] * X_list[s][j]
        ll.append(p)

    odds.append(sum(ll))

print("Xi :", Xi)
print("Xixi :", XiXi)
print("odds :", odds)

matrix_content = [100.0, 505.0, 554.0457727056, 606.1264987875, XiXi[0], odds[0], odds[2], XiXi[1], odds[1], XiXi[2]]

Xi : [505.0, 554.0457727056, 606.1264987875]
Xixi : [3383.5, 3929.6827839206667, 4472.48767614537]
odds : [3641.01643904243, 4171.374220465154, 3863.42711949835]


In [303]:
Matrix_X = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

# Fill the upper and lower triangular parts symmetrically
index = 0
for i in range(k):
    for j in range(i, k):
        Matrix_X[i][j] = matrix_content[index]
        Matrix_X[j][i] = matrix_content[index]
        index += 1
for row in Matrix_X:
    print(row)

[100.0, 505.0, 554.0457727056, 606.1264987875]
[505.0, 3383.5, 3641.01643904243, 3863.42711949835]
[554.0457727056, 3641.01643904243, 3929.6827839206667, 4171.374220465154]
[606.1264987875, 3863.42711949835, 4171.374220465154, 4472.48767614537]


In [304]:
# A_Transpose = np.transpose(A) or A.T #Here A should be np.array([[], [], []])
# New_X is a symmetrix matrix so It's transpose should be same as X.
Matrix_X_Transpose = Matrix_X

# Matrix multiplication :
# numpy.matmul(A, B) ; Here A, B need not be np.array, Simple 2-D list.
# A @ B @ C  ; Here A, B need to be np.array and First it will multiply (AXB) then (result X C).
Matrix_A = np.matmul(Matrix_X_Transpose, Matrix_X)

# Inverse of matrix A where A should not be singular.
Matrix_A_inverse = np.linalg.inv(Matrix_A)

result = np.matmul(np.matmul(Matrix_A_inverse, Matrix_X_Transpose), Vector_Y)

print("value of beta0 :", result[0])
print("value of beta1 :", result[1])
print("value of beta2 :", result[2])
print("value of beta3 :", result[3])

value of beta0 : 13.239477912194616
value of beta1 : 6.132437615571689
value of beta2 : 2.392265550166485
value of beta3 : 7.746810380405009


In [305]:
predicted_Y = []
for i in range(100):
  r1 = result[0] + result[1] * X1[j] + result[2] * X2[j] + result[3] * X3[j]
  predicted_Y.append(r1)

# Verification

In [307]:
df.head()

,x1,x2,x3,y
0,0.1,0.311089,0.390541,14.705876
1,0.2,0.639066,1.389918,26.585100
2,0.3,0.840228,1.939903,36.791509
3,0.4,1.299710,2.153009,34.090806
4,0.5,0.941784,0.945136,31.118089


In [308]:
X = df[['x1', 'x2', 'x3']]
Y = df['y']

In [309]:
regr = LinearRegression()

regr.fit(X, Y)
print("Slope: ",regr.coef_)
print("Intercept: ",regr.intercept_)

Slope:  [6.13243763 2.39226554 7.74681038]
Intercept:  13.239477824445373


In [310]:
#Now predicting the value of every data.
# x_array = np.array(X).reshape(-1, 1)
y_array = np.array(Y).reshape(-1, 1)

y_pred_array = regr.predict(X)
y_pred_list = list(y_pred_array.reshape(1,-1)[0])

In [314]:
# Error between "prediction by SKLearn Regression model" and "given data."

mae = mean_absolute_error(y_true=Y,y_pred=y_pred_list)
mse = mean_squared_error(y_true=Y,y_pred=y_pred_list)
rmse = root_mean_squared_error(y_true=Y,y_pred=y_pred_list)

print("MAE:",mae)
print("MSE:",mse)
print("RMSE:",rmse)


MAE: 5.155505630378741
MSE: 34.62048082924355
RMSE: 5.8839171331047435


In [315]:
# Error between "prediction by My Own Regression model" and "given data."

mae = mean_absolute_error(y_true=Y,y_pred=predicted_Y)
mse = mean_squared_error(y_true=Y,y_pred=predicted_Y)
rmse = root_mean_squared_error(y_true=Y,y_pred=predicted_Y)

print("MAE:",mae)
print("MSE:",mse)
print("RMSE:",rmse)

MAE: 69.95183669407393
MSE: 6940.052450086543
RMSE: 83.30697719931112
